# Practice Questions - 08-29-2025

In [5]:
import os
import sqlite3
import pandas as pd

In [2]:
os.getcwd()

'/Users/neilbardhan/Documents/python-projects/technical-interview-prep/data/notebooks'

In [4]:
os.chdir('..')

## Connect to Database

In [7]:
def print_tables(connection):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(tables)
    cursor.close()

In [10]:
DB_NAME = 'ecommerce.db'
DB_PATH = "{}/{}".format(os.getcwd(), "datasets/")
db_file_path = "{}{}".format(DB_PATH, DB_NAME)

In [12]:
if os.path.isfile(db_file_path):
    connection = sqlite3.connect(db_file_path)
    print_tables(connection)

[('customer',), ('product',), ('transactions',), ('transaction_items',), ('reviews',)]


### Print Schemas of Tables

In [37]:
def get_table_schema(connection, table_name):
    pragma_query = "PRAGMA table_info({});".format(table_name)
    print(pragma_query)
    cursor = connection.cursor()
    cursor.execute(pragma_query)
    pragma_info = cursor.fetchall()
    for row in pragma_info:
        print("{}:\t{}".format(row[1], row[2]))
    cursor.close()

In [43]:
get_table_schema(connection, "customer")

PRAGMA table_info(customer);
customer_id:	INTEGER
name:	TEXT
email:	TEXT
signup_date:	TEXT
country:	TEXT


In [44]:
get_table_schema(connection, "product")

PRAGMA table_info(product);
product_id:	INTEGER
name:	TEXT
category:	TEXT
price:	REAL


In [45]:
get_table_schema(connection, "transactions")

PRAGMA table_info(transactions);
order_id:	INTEGER
customer_id:	INTEGER
order_date:	TEXT
total_amount:	REAL


In [47]:
get_table_schema(connection, "transaction_items")

PRAGMA table_info(transaction_items);
order_item_id:	INTEGER
order_id:	INTEGER
product_id:	INTEGER
quantity:	INTEGER
item_price:	REAL


In [48]:
get_table_schema(connection, "reviews")

PRAGMA table_info(reviews);
review_id:	INTEGER
order_id:	INTEGER
product_id:	INTEGER
rating:	INTEGER
review_date:	TEXT


## Practice Questions

In [49]:
def query_runner(connection, query):
    query_df = pd.read_sql_query(query, connection)
    return query_df

In [50]:
customer_table_all_query = """
SELECT *
FROM customer;
"""
query_results_df = query_runner(connection, customer_table_all_query)
query_results_df.head()

,customer_id,name,email,signup_date,country
0,1,Raymond Conley,vking@example.com,2025-03-14,Malaysia
1,2,Robert Andersen,crystal34@example.org,2024-03-14,Niger
2,3,Jody Martinez,jlane@example.com,2023-12-15,Jamaica
3,4,Kenneth Shepherd,kimberlypeters@example.net,2023-07-25,Peru
4,5,Carolyn Taylor,joshua08@example.org,2025-03-13,Mayotte


### List the first 10 customers who signed up in 2022. Show `customer_id, name, signup_date`.

In [54]:
sql_query = """
SELECT signup_date, typeof(signup_date)
FROM customer;
"""
query_results_df = query_runner(connection, sql_query)
query_results_df.head()

,signup_date,typeof(signup_date)
0,2025-03-14,text
1,2024-03-14,text
2,2023-12-15,text
3,2023-07-25,text
4,2025-03-13,text
